In [0]:
from google.colab import files
files.upload()

In [0]:
 ! mkdir ~/.kaggle 

In [0]:

! cp kaggle.json ~/.kaggle/

In [0]:
! chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d pmigdal/alien-vs-predator-images

In [0]:
! mkdir alien-vs-predator

In [0]:
! unzip alien-vs-predator-images.zip -d alien-vs-predator

In [0]:
from keras.applications.resnet50 import ResNet50, preprocess_input

In [0]:
Height,Width=300,300
base_model=ResNet50(weights='imagenet',include_top=False,input_shape=(Height,Width,3))

In [0]:
train_dir='/content/alien-vs-predator/data/train'
batch_size=16

In [0]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=90,horizontal_flip=True,vertical_flip=True,shear_range=0.2)

In [0]:
train_generator=train_datagen.flow_from_directory(train_dir,target_size=(Height,Width),batch_size=batch_size)

In [0]:
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.models import Sequential,Model

In [0]:
def build_finetune_model(base_model,dropout,fc_layers,num_classes):
  for layer in base_model.layers:
    layer.trainable=False
  x=base_model.output
  x=Flatten()(x)
  for fc in fc_layers:
    x=Dense(fc,activation='relu')(x)
    x=Dropout(dropout)(x)

  predictions=Dense(num_classes,activation='sigmoid')(x)
  finetune_model=Model(inputs=base_model.input,outputs=predictions)
  return finetune_model

In [0]:
class_list=['alien','predators']
fc_layers=[1024,1024]
dropout=0.5

In [0]:
finetune_model=build_finetune_model(base_model,dropout=dropout,fc_layers=fc_layers,num_classes=len(class_list))

In [0]:
from keras.optimizers import SGD,Adam
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt


In [0]:
num_epochs=10
batch_size=16
num_training_images=694

In [0]:
adam=Adam(learning_rate=0.00001)
finetune_model.compile(adam,loss='binary_crossentropy',metrics=['accuracy'])
filepath="./checkpoints/"+"ResNet50"+"_model_weights.h5"
checkpoint=ModelCheckpoint(filepath=filepath,monitor=["acc"],verbose=1,mode="max")

In [0]:
!mkdir checkpoints

In [0]:
callback_list=[checkpoint]
history=finetune_model.fit_generator(train_generator,epochs=num_epochs,steps_per_epoch=num_training_images//batch_size,shuffle=True,callbacks=callback_list)

In [0]:
def plot_training(history):
  acc=history.history['accuracy']
  print(history.history)
  #val_acc=history.history['val_acc']
  loss=history.history['loss']
  #val_loss=history.history['val_loss']
  epochs=range(len(acc))
  plt.plot(epochs,acc,'r.')
  #plt.plot(epochs,val_acc,'r')
  plt.title('Training and validation accuracy')
  plt.show()

In [0]:
plot_training(history)